In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu' #this is to show if cuda is available on the device being used.
print(device)
# import torch
# device = 'cpu'
# print(device)
max_iters=1000
# eval interval = 2500
learning_rate = 3e-4
eval_iters = 250
#dropout = 0.2 is for dropping certain neurons to make model performance more accurate and avoid noises due to excessive neurons.


cpu


C:\Users\admin\Desktop\Language Model\cuda\lib\site-packages\torch\cuda\__init__.py:118: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 6050). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\cuda\CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
with open('wiz_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print(len(text))

# Extract unique characters from the text
chars = sorted(set(text))

string_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_string = {i: ch for i, ch in enumerate(chars)}

# Function to encode a string
def encode(s):
    return [string_to_int[c] for c in s]

# Function to decode a list of integers
def decode(l):
    return ''.join([int_to_string[i] for i in l if i in int_to_string])


#just for testing
# encoded_hello = encode('hello')
# decoded_hello = decode(encoded_hello)
# print(encoded_hello)
# print(decoded_hello)

232307


In [3]:
data = torch.tensor(encode(text), dtype = torch.long)
print(data[:100])

tensor([80, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,  0,
         0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,  0,
         1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47, 33,
        50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36, 25,
        38, 28,  1, 39, 30,  1, 39, 50,  9,  1])


In [4]:
block_size = 8
batch_size = 4
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[:n]

In [5]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    #print(ix)
    x= torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    #x, y = x.to(device, y.to(device))
    return x, y

x, y = get_batch('train')
print('inputs:')
#print(x.shape)
print(x)
print('targets: ')
print(y)

x = train_data[:block_size]
y = train_data[1:block_size+1]

inputs:
tensor([[68, 74,  1, 71, 58, 66, 58, 66],
        [65, 68, 76,  1, 76, 54, 72,  1],
        [56, 73,  1, 76, 54, 72,  1, 73],
        [59, 58, 76,  1, 57, 68, 79, 58]])
targets: 
tensor([[74,  1, 71, 58, 66, 58, 66, 55],
        [68, 76,  1, 76, 54, 72,  1, 65],
        [73,  1, 76, 54, 72,  1, 73, 61],
        [58, 76,  1, 57, 68, 79, 58, 67]])


In [6]:
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is ', context, 'target is ', target)

when input is  tensor([80]) target is  tensor(28)
when input is  tensor([80, 28]) target is  tensor(39)
when input is  tensor([80, 28, 39]) target is  tensor(42)
when input is  tensor([80, 28, 39, 42]) target is  tensor(39)
when input is  tensor([80, 28, 39, 42, 39]) target is  tensor(44)
when input is  tensor([80, 28, 39, 42, 39, 44]) target is  tensor(32)
when input is  tensor([80, 28, 39, 42, 39, 44, 32]) target is  tensor(49)
when input is  tensor([80, 28, 39, 42, 39, 44, 32, 49]) target is  tensor(1)


In [7]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None): #normal forward fnx for every model phase building created and defined
        logits = self.token_embedding_table(index)
        if targets is None:
            loss=None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    
    def generate(self, index, max_new_tokens): #normal generate function for every model phase building created and defined
        #index is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            #get the predictions
            logits, loss = self.forward(index)
            #focus only on the last time step
            logits = logits[:, -1, :] # becomes (B,C)
            #apply softmax to get probabilities
            probs = F.softmax(logits, dim =-1)# (B,C)
            #sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B,1)
            #append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index


In [10]:
# Instantiate the model
vocab_size = 10000  # Example value, replace with actual vocab size which is \
                    # 40,000 for my dataset, yours may be different with respect to your data.\
                    # I used 10,000 because my laptop is very poor in computationa power and also uses a cpu.
model = BigramLanguageModel(vocab_size)

In [11]:
# Define context and generate text
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = model.generate(context, max_new_tokens=500)
print(generated_chars)

tensor([[   0, 6392, 6336, 8434,   49, 4141, 3431, 8421, 2070, 8282,  141, 4602,
         5737, 7911, 2718, 3440, 1100,   71, 3674, 4730, 2617,  304, 2368, 1603,
         2434, 6250, 6831, 3068, 6664, 2961, 3665, 7744, 2755,  411, 6854, 8036,
         1149, 1753, 5079, 1034, 6416, 4524, 2408, 1977, 6101, 6318, 8458, 3446,
         8986, 3823, 5563, 2273, 6890, 3998, 5643, 8111, 6090, 6381, 7692, 6947,
         3048, 6326, 2758, 3479, 3830, 6453, 7786, 3416, 7207, 4969, 2861, 8366,
         7542, 1714,  781, 9178, 6297, 9157, 4439, 5804, 9658, 2119, 6174, 4586,
         3802, 6108, 3234, 6271, 3479, 6217, 2527, 3979, 1214, 8080, 7394, 1495,
         4857, 6296,  216, 5194, 6147, 1473, 8419, 3719, 2032, 8361, 1645, 5090,
         7622, 1764, 8699, 1095, 7855, 3717, 4119, 8736, 1902, 6648, 9098, 5244,
         7465, 8364, 2129, 9685, 7323, 6289, 5111, 3408, 1096, 3603, 5067, 2913,
         9302, 2990, 9004, 9180, 8376, 8524, 5524, 1102, 8809, 5340, 6812, 9684,
         1127, 2352, 9409,  

In [ ]:
#creating a pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    
#     if iter % eval_iters ==0:
#         #something soon now...
#         print(f'step: {iter}, loss{losses}')
    
    #sample a batch of data
    xb, yb = get_batch('train')
    
    #evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

In [ ]:
context = torch.zeros((1,1), dtype=torch.long, device = device)
generated_chars = decode(model.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

# #there's this no_grad way for training, it enables space saving in storage, training without gradient computations.
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        X, Y = get_batch(split)
        logits, loss=model(X,Y)
        losses[k] = loss.item()
        out[spli]=losses.mean()
    model.train()
    return out